In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from utils import *
import pandas as pd

# 数据预处理
data = np.load('data.npz')
train_data = data['train_data']
test_data = data['test_data']
train_labels = data['train_labels']

train_data.shape, train_labels.shape, test_data.shape

((15000, 60, 8), (15000,), (16000, 60, 8))

In [2]:
train_data = tf.cast(train_data, tf.float32).numpy()
train_labels = tf.one_hot(train_labels, 20).numpy()

In [3]:
# 定义注意力模块
def se_module(seq, reduction_ratio = 16):
    #[None, time_steps, channels]
    se = keras.layers.GlobalAveragePooling1D()(seq)
    #[None, channels]
    se = keras.layers.Dense(seq.shape[-1] / reduction_ratio, activation='relu')(se)
    se = keras.layers.Dense(seq.shape[-1], activation='relu')(se)
    se = tf.nn.sigmoid(se)
    #[None, channels]
    se = keras.layers.Reshape([1, seq.shape[-1]])(se)
    #[None, 1, channels]
    return tf.multiply(seq, se)

In [4]:
def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape[1:])
    masking = keras.layers.Masking(mask_value=0.0)(inputs)
    seq = keras.layers.LSTM(128, return_sequences=True,
                                unit_forget_bias=False,
                                recurrent_regularizer=keras.regularizers.l2(0.03),
    )(masking)
    seq = keras.layers.LayerNormalization()(seq)
    seq = se_module(seq)
    forward_layer = keras.layers.LSTM(128, return_sequences=True,
                        unit_forget_bias=False,
                        recurrent_regularizer = keras.regularizers.l2(0.015),
                        )
    backward_layer = keras.layers.LSTM(128, return_sequences=True,
                        go_backwards=True,
                        unit_forget_bias=False,
                        recurrent_regularizer = keras.regularizers.l2(0.005),
                        )
    seq = keras.layers.Bidirectional(
                        layer=forward_layer,
                        backward_layer=backward_layer,
                        )(seq)
    seq = keras.layers.GlobalAveragePooling1D()(seq)
    seq = keras.layers.Dense(512, activation='relu')(seq)
    seq = keras.layers.Dropout(0.3)(seq)
    seq = keras.layers.Dense(128, activation='relu')(seq)
    seq = keras.layers.Dropout(0.3)(seq)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(seq)

    model = keras.models.Model(inputs=[inputs], outputs=[outputs])

    model.compile(optimizer=tf.optimizers.Adam(),
            loss=tf.losses.CategoricalCrossentropy(label_smoothing=0.1),           
            metrics=['accuracy'])
    return model

In [5]:
def scheduler_func(epoch):
    if epoch < 15:
        rate = 1e-3
    elif epoch >= 15 and epoch < 25:
        rate = 3e-4
    elif epoch >= 25:
        rate = 9e-5
    return rate * 1.0

In [6]:
kfcv_fit(builder=lambda : build_model(train_data.shape, 20),
                x=train_data, y=train_labels,
                epochs=40,
                checkpoint_path = './models/lstm/',
                batch_size=64,
                extra_callbacks=[keras.callbacks.LearningRateScheduler(scheduler_func)],
                verbose=2,
                noise_std=0.05
                )

99 - val_loss: 1.1214 - val_accuracy: 0.8133 - lr: 9.0000e-05
Epoch 33/40
446/446 - 9s - loss: 0.7220 - accuracy: 0.9802 - val_loss: 1.1170 - val_accuracy: 0.8200 - lr: 9.0000e-05
Epoch 34/40
446/446 - 9s - loss: 0.7224 - accuracy: 0.9789 - val_loss: 1.1223 - val_accuracy: 0.8120 - lr: 9.0000e-05
Epoch 35/40
446/446 - 9s - loss: 0.7193 - accuracy: 0.9803 - val_loss: 1.1171 - val_accuracy: 0.8200 - lr: 9.0000e-05
Epoch 36/40
446/446 - 9s - loss: 0.7176 - accuracy: 0.9815 - val_loss: 1.1180 - val_accuracy: 0.8187 - lr: 9.0000e-05
Epoch 37/40
446/446 - 9s - loss: 0.7167 - accuracy: 0.9807 - val_loss: 1.1221 - val_accuracy: 0.8200 - lr: 9.0000e-05
Epoch 38/40
446/446 - 9s - loss: 0.7157 - accuracy: 0.9823 - val_loss: 1.1177 - val_accuracy: 0.8147 - lr: 9.0000e-05
Epoch 39/40
446/446 - 9s - loss: 0.7127 - accuracy: 0.9824 - val_loss: 1.1174 - val_accuracy: 0.8187 - lr: 9.0000e-05
Epoch 40/40
446/446 - 9s - loss: 0.7125 - accuracy: 0.9826 - val_loss: 1.1211 - val_accuracy: 0.8200 - lr: 9.000

In [7]:
y_pred = kfcv_predict('models/lstm', test_data)
np.save('lstm_20fold (without_feature1).npy', y_pred)

models/lstm loaded.
result got


In [8]:
submit_example = pd.read_csv('dataset/submit_example.csv')
submit_example['behavior_id'] = np.argmax(y_pred, axis=-1)
submit_example.to_csv('submit.csv', index=None)

In [10]:
from score_estimator import *
test()

,0.7325.csv,0.7453.csv,0.7502.csv,0.7514.csv,0.7524.csv,0.7566.csv,0.7589.csv,0.7590.csv,0.7620.csv,0.7635.csv,0.7636.csv,0.7810.csv
acc,0.804375,0.871125,0.770938,0.876687,0.864875,0.823125,0.839625,0.838375,0.832875,0.831063,0.777125,0.877375
acc_combo,0.825500,0.884723,0.793982,0.889286,0.878452,0.842241,0.855851,0.855854,0.850741,0.849589,0.800092,0.890259
